In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [4]:

# ---------------------------
# 1️⃣ Veriyi yükleme ve normalize etme
# ---------------------------
transform = transforms.Compose([
    transforms.ToTensor(),           # 0–255 → 0–1 arası
    transforms.Lambda(lambda x: x.view(-1))  # 28x28 → 784
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)


100.0%
100.0%
100.0%
100.0%


In [17]:

# ---------------------------
# 2️⃣ Basit MLP modeli
# ---------------------------
model = nn.Sequential(
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 10)   # 10 sınıf
)

# ---------------------------
# 3️⃣ Loss ve optimizer
# ---------------------------
criterion = nn.CrossEntropyLoss()   # softmax + cross-entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [18]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
start.record()
# ---------------------------
# 4️⃣ Eğitim döngüsü
# ---------------------------
epochs = 5
device = torch.device("cpu") #torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Doğruluk
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}, Accuracy: {train_acc:.2f}%")

end.record()
torch.cuda.synchronize()
print(f"Training time: {start.elapsed_time(end):.2f} ms")

Epoch 1/5, Loss: 0.3423, Accuracy: 90.21%
Epoch 2/5, Loss: 0.1436, Accuracy: 95.77%
Epoch 3/5, Loss: 0.0970, Accuracy: 97.03%
Epoch 4/5, Loss: 0.0725, Accuracy: 97.80%
Epoch 5/5, Loss: 0.0565, Accuracy: 98.22%
Training time: 30530.72 ms


In [19]:
device

device(type='cpu')

In [20]:

# ---------------------------
# 5️⃣ Test doğruluğu
# ---------------------------
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 97.39%


In [ ]:
#gerçek ve tahmin edilen ilk 30 örneği yazdırma
model.eval()
with torch.no_grad():
    images, labels = next(iter(test_loader))
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)
    print("Gerçek: ", labels[:30].cpu().numpy())
    print("Tahmin: ", predicted[:30].cpu().numpy())


Gerçek:  [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1]
Tahmin:  [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1]
